In [1]:
%matplotlib inline
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SQLContext
from datetime import *
from dateutil.parser import parse

In [2]:
sc

In [83]:
"""
SqlDate: 1
Date, Month: need to compute
Year: 3

IsRootEvent	25
EventCode	26
EventBaseCode	27
EventRootCode	28
QuadClass	29
GoldsteinScale	30
NumArticles	33
AvgTone	34


ActionGeo_CountryCode 51
"""
def parseRec(strRecord):
    
    if strRecord is None:
        return None
    
    try:
        rec = strRecord.split('\t')
        
        if len(rec)<57:
            return None
        
        globalEventId = int(rec[0])
        
        isRootEvent = rec[25]        
        if (isRootEvent is None) or (int(isRootEvent) == 0):
            return None

        sqlDate = rec[1]
        if (sqlDate is None or len(sqlDate)==0):
            return None        
        eventDate = parse(sqlDate).date()
        eventMonth = eventDate.month
        
        if (rec[3] is None or len(rec[3])==0):
            return None
        eventYear= int(rec[3])
        
        if (rec[26] is None or len(rec[26])==0):
            return None
        eventCode= int(rec[26])
        
        if (rec[27] is None or len(rec[27])==0):
            return None
        eventBaseCode= int(rec[27])        
        
        if (rec[28] is None or len(rec[28])==0):
            return None
        eventRootCode= int(rec[28])  
                
        if (rec[29] is None or len(rec[29])==0):
            return None
        quadClass= int(rec[29])          
         
        if (rec[30] is None or len(rec[30])==0):
            return None
        goldsteinScale= float(rec[30])  
         
        if (rec[33] is None or len(rec[33])==0):
            return None
        numArticles= int(rec[33])
         
        if (rec[34] is None or len(rec[34])==0):
            return None
        avgTone= float(rec[34])
        
        if (rec[51] is None or len(rec[51])==0):
            return None
        actionGeoCountryCode = rec[51]
        
        return (globalEventId, eventDate, eventMonth, eventYear, eventCode, eventBaseCode, eventRootCode, quadClass, goldsteinScale, numArticles, avgTone, actionGeoCountryCode)
    
    except:
        return None
    
    

parsedRdd = sc.textFile("s3://gdeltdata/data/*").map(lambda x: parseRec(x)).filter(lambda x: x is not None)
#yCabRDD = yCabRDD.map(lambda record: yCabParse(record))

In [84]:
parsedRdd.cache()

PythonRDD[45] at RDD at PythonRDD.scala:43

In [85]:
%time parsedRdd.count()

CPU times: user 244 ms, sys: 96 ms, total: 340 ms
Wall time: 14min 5s


221411551

In [56]:
%time parsedRdd.count()

CPU times: user 268 ms, sys: 76 ms, total: 344 ms
Wall time: 13min 57s


228601011

In [86]:
#globalEventId, eventDate, eventMonth, eventYear, eventCode, eventBaseCode, eventRootCode, quadClass, goldsteinScale, 
#numArticles, avgTone, actionGeoCountryCode)
def toCSVLine(record):   
    return ','.join(str(d) for d in record)

csvRDD = parsedRdd.map(toCSVLine)
#csvRDD.repartition(7).saveAsTextFile('s3://gdeltdata/coredata')

In [89]:
csvRDD.repartition(7).saveAsTextFile('s3://gdeltdata/coredata')